In [1]:
import pandas as pd

In [41]:
df = pd.read_csv('kaggle_poem_dataset.csv')
df = df[(df['Content'].str.count('\n') < 8)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1106 entries, 0 to 15650
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            1106 non-null   int64 
 1   Author                1106 non-null   object
 2   Title                 1106 non-null   object
 3   Poetry Foundation ID  1106 non-null   int64 
 4   Content               1106 non-null   object
dtypes: int64(2), object(3)
memory usage: 38.9+ KB
